# Анализ тональности отзывов (соревнование на kaggle.com) 

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize  
from nltk.corpus import sentiwordnet as swn

from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV

### Загрузка данных для обучения и предсказания:

In [2]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None, names=['text', 'target'])
test = pd.read_csv('products_sentiment_test.tsv', sep='\t', index_col='Id')

### Выполняем преобразования текстов для лучшей работы классификаторов. Удаляем стоп слова, проводим лемматизацию.

In [5]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize_wtag(notlemmaword):
    for word, tag in pos_tag(word_tokenize(notlemmaword)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, wntag)
        return lemma

def text_transform(line):
    words = [x.strip() for x in line.lower().split()]
    result = []
    for word in words:
        word = lemmatize_wtag(word)
        result.append(word)      
    return ' '.join(result)

In [6]:
train['text_transformated'] = train.text.apply(text_transform)

In [7]:
test['text_transformated'] = test.text.apply(text_transform)

In [8]:
train.head()

text  target  \
0          2 . take around 10,000 640x480 pictures .       1   
1  i downloaded a trial version of computer assoc...       1   
2  the wrt54g plus the hga7t is a perfect solutio...       1   
3  i dont especially like how music files are uns...       0   
4  i was using the cheapie pail ... and it worked...       1   

                                  text_transformated  
0           2 . take around 10,000 640x480 picture .  
1  i download a trial version of computer associa...  
2  the wrt54g plus the hga7t be a perfect solutio...  
3  i dont especially like how music file be unstr...  
4  i be use the cheapie pail ... and it work ok u...

### Выбираем классификатор:

In [17]:
def score(clf, train, target):
    scores = cross_val_score(clf, train, target, cv=5)
    print("CLF score mean = {:.7f}, std = {:.7f}".format(scores.mean(), scores.std()))

In [23]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

def save_predicts(predicts, fname):
    df = pd.DataFrame(predicts, columns=['y'])
    df.to_csv(fname, header=True, index_label='Id')

In [18]:
#countvectorizer с необработанным текстом
for classificator in [PassiveAggressiveClassifier, Perceptron, LogisticRegression, 
                      SGDClassifier, LinearSVC, MultinomialNB, BernoulliNB]:
    clf = make_pipeline(CountVectorizer(ngram_range=(1,1)), classificator())
    score(clf, train.text, train.target)

CLF score mean = 0.7364994, std = 0.0146341
CLF score mean = 0.7329956, std = 0.0239894
CLF score mean = 0.7684957, std = 0.0076341
CLF score mean = 0.7239542, std = 0.0379058
CLF score mean = 0.7540007, std = 0.0136402
CLF score mean = 0.7789820, std = 0.0190816
CLF score mean = 0.7430006, std = 0.0057472


In [19]:
#TfidfVectorizer с необработанным текстом
for classificator in [PassiveAggressiveClassifier, Perceptron, LogisticRegression, 
                      SGDClassifier, LinearSVC, MultinomialNB, BernoulliNB]:
    clf = make_pipeline(TfidfVectorizer(ngram_range=(1,1)), classificator())
    score(clf, train.text, train.target)

CLF score mean = 0.7624882, std = 0.0119502
CLF score mean = 0.7404931, std = 0.0083260
CLF score mean = 0.7665032, std = 0.0110669
CLF score mean = 0.7624769, std = 0.0171404
CLF score mean = 0.7684857, std = 0.0144218
CLF score mean = 0.7070030, std = 0.0071748
CLF score mean = 0.7430006, std = 0.0057472


In [20]:
#CountVectorizer с необработанным текстом
for classificator in [PassiveAggressiveClassifier, Perceptron, LogisticRegression, 
                      SGDClassifier, LinearSVC, MultinomialNB, BernoulliNB]:
    clf = make_pipeline(CountVectorizer(ngram_range=(1,1)), classificator())
    score(clf, train.text_transformated, train.target)

CLF score mean = 0.7610144, std = 0.0119310
CLF score mean = 0.7429994, std = 0.0193293
CLF score mean = 0.7750032, std = 0.0095387
CLF score mean = 0.7515094, std = 0.0076895
CLF score mean = 0.7505094, std = 0.0095619
CLF score mean = 0.7840070, std = 0.0117675
CLF score mean = 0.7549994, std = 0.0103752


In [21]:
#TfidfVectorizer с обработанным текстом
for classificator in [PassiveAggressiveClassifier, Perceptron, LogisticRegression, 
                      SGDClassifier, LinearSVC, MultinomialNB, BernoulliNB]:
    clf = make_pipeline(TfidfVectorizer(ngram_range=(1,1)), classificator())
    score(clf, train.text_transformated, train.target)

CLF score mean = 0.7600157, std = 0.0157546
CLF score mean = 0.7399943, std = 0.0101459
CLF score mean = 0.7705069, std = 0.0127626
CLF score mean = 0.7555157, std = 0.0137433
CLF score mean = 0.7729857, std = 0.0135742
CLF score mean = 0.7174993, std = 0.0122559
CLF score mean = 0.7549994, std = 0.0103752


#### Возьмем LogisticRegression и CountVectorizer и подберем гиперпарамметры через GridSearchCV

In [22]:
%%time
# с необработанными данными

param_grid = {
    'logisticregression__penalty': ('l1', 'l2'),
    'logisticregression__C': (1, 2),
    'countvectorizer__max_df': (0.7, 0.85, 1.0),
    'countvectorizer__ngram_range': ((1, 1), (1, 2))
}

cv = StratifiedShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)

model = make_pipeline(CountVectorizer(tokenizer=LemmaTokenizer()), LogisticRegression())
rsearch = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=cv, error_score=0)

rsearch.fit(train.text, train.target)
print (rsearch.best_score_)
print (rsearch.best_params_)


0.7885
{'countvectorizer__ngram_range': (1, 2), 'logisticregression__penalty': 'l2', 'countvectorizer__max_df': 0.7, 'logisticregression__C': 2}
CPU times: user 2.78 s, sys: 270 ms, total: 3.05 s
Wall time: 1min 47s


In [24]:
save_predicts(rsearch.predict(test.text), 'submiss01.csv') #на kaggle 0.80000

In [25]:
%%time
# с обработанными данными

param_grid = {
    'logisticregression__penalty': ('l1', 'l2'),
    'logisticregression__C': (1, 2),
    'countvectorizer__max_df': (0.7, 0.85, 1.0),
    'countvectorizer__ngram_range': ((1, 1), (1, 2))
}

cv = StratifiedShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)

model = make_pipeline(CountVectorizer(tokenizer=LemmaTokenizer()), LogisticRegression())
rsearch = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=cv, error_score=0)

rsearch.fit(train.text_transformated, train.target)
print (rsearch.best_score_)
print (rsearch.best_params_)

0.7915
{'countvectorizer__ngram_range': (1, 2), 'logisticregression__penalty': 'l2', 'countvectorizer__max_df': 0.7, 'logisticregression__C': 2}
CPU times: user 2.98 s, sys: 277 ms, total: 3.26 s
Wall time: 1min 57s


In [35]:
save_predicts(rsearch.predict(test.text_transformated), 'submiss02.csv') #на kaggle 0.78250

#### Возьмем MultinomialNB и CountVectorizer и подберем гиперпарамметры через GridSearchCV

In [29]:
%%time
# с необработанными данными

param_grid = {
    'multinomialnb__alpha': [0.01,0.05,0.1,0.4,0.5,0.7,0.9,1.0,1.2,1.5,2.0],
    'countvectorizer__max_df': (0.7, 0.85, 1.0),
    'countvectorizer__ngram_range': ((1, 1), (1, 2))
}

cv = StratifiedShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)

model = make_pipeline(CountVectorizer(tokenizer=LemmaTokenizer()), MultinomialNB())
rsearch = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=cv, error_score=0)

rsearch.fit(train.text, train.target)
print (rsearch.best_score_)
print (rsearch.best_params_)

0.8015
{'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 0.4, 'countvectorizer__max_df': 0.7}
CPU times: user 4.97 s, sys: 576 ms, total: 5.55 s
Wall time: 4min 45s


In [30]:
save_predicts(rsearch.predict(test.text), 'submiss03.csv') #на kaggle 0.79000

In [31]:
%%time
# с обработанными данными

param_grid = {
    'multinomialnb__alpha': [0.01,0.05,0.1,0.4,0.5,0.7,0.9,1.0,1.2,1.5,2.0],
    'countvectorizer__max_df': (0.7, 0.85, 1.0),
    'countvectorizer__ngram_range': ((1, 1), (1, 2))
}

cv = StratifiedShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)

model = make_pipeline(CountVectorizer(tokenizer=LemmaTokenizer()), MultinomialNB())
rsearch = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1, cv=cv, error_score=0)

rsearch.fit(train.text_transformated, train.target)
print (rsearch.best_score_)
print (rsearch.best_params_)

0.8075
{'countvectorizer__ngram_range': (1, 1), 'multinomialnb__alpha': 0.7, 'countvectorizer__max_df': 1.0}
CPU times: user 5.35 s, sys: 576 ms, total: 5.93 s
Wall time: 4min 41s


In [34]:
save_predicts(rsearch.predict(test.text_transformated), 'submiss04.csv') #на kaggle 0.78250